In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append("../dataprocess")
import dataprocess as dp

#### Parameters

In [5]:
# Mesh
nx = 227  # X-direction nodes
ny = 120  # Y-direction nodes
# Domain:
Lx = 17.665369  # Lx (Size of the box in x-direction)
Ly = 9.0  # Ly (Size of the box in y-direction)
# Cylinder coordinates:
X_0 = 2.66537  # X coordinate of the center
Y_0 = 4.5  # Y coordinate of the center
r = 0.5  # Cylinder radius

mesh = dp.Mesh(nx, ny, Lx, Ly, X_0, Y_0, r)

# Data path
re = 50
train_input_dir = "../data/modV_train/"
input_filename = f'modV_crop_re{re}.csv'

#### Data preprocessing

In [6]:
''' Read velocity data
  X - velocity module
 rows of X correspond to velocity components at spatial locations
 columns of X correspond to timesteps
     t_1 t_2.  .  t_n
 X = [u  u  .  .  .]  (x_1,y_1)
     [v  v  .  .  .]  (x_1,y_1)
     [w  w  .  .  .]  (x_1,y_1)
     [u  u  .  .  .]  (x_2,y_2)
     [v  v  .  .  .]  (x_2,y_2)
     [w  w  .  .  .]  (x_2,y_2)
     [.  .  .  .  .]   .
     [.  .  .  .  .]   .
     [.  .  .  .  .]   .
'''
X_train = dp.read_X_csv(os.path.join(train_input_dir, input_filename))
# Scale data
std_scaler = StandardScaler()
X_train = std_scaler.fit_transform(X_train)

n, m = X_train.shape

print("Data matrix X is n by m:", n, "x", m, flush=True)

# Create mesh class instance
mesh = dp.Mesh(nx, ny, Lx, Ly, X_0, Y_0, r)
# Erase cylinder from snapshot
X_train_filt = dp.erase_cyl(X_train, mesh)

Reading data from: ../data/modV_train/modV_crop_re50.csv
Data matrix X is n by m: 27240 x 1000
Snapshot points without cylinder (27104,)


In [2]:
# Check if GPU can be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Running on GPU")
    print(torch.cuda.get_device_name(device))
else:
    print("Running on CPU")

Running on CPU


In [8]:
# Prepare dataset for pyTorch
# default data arrangement in pytorch is different than ours, so we have to transpose X
X_tensor = torch.from_numpy(X_train_filt.T)
dataset = torch.utils.data.TensorDataset(X_tensor)
batchsize = 128
# Set seed for reproducible results
seed = 42
torch.manual_seed(seed)
# shuffle data manually and save indices
index_list = torch.randperm(len(dataset)).tolist()
shuffled_dataset = torch.utils.data.Subset(dataset, index_list)
data_loader = torch.utils.data.DataLoader(
    shuffled_dataset, batch_size=batchsize, shuffle=False
)

#### Autoencoder

In [9]:
# Define autoencoder network structure
class Autoencoder_Linear(nn.Module):
    def __init__(self, n):

        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n, 8192),
            nn.ReLU(),
            nn.Linear(8192, 2048),
            nn.ReLU(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 8),
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 2048),
            nn.ReLU(),
            nn.Linear(2048, 8192),
            nn.ReLU(),
            nn.Linear(8192, n),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
# Define the model
model = Autoencoder_Linear(n).to(device)

# Try to load the training coefficients from the .pt file if it exists
# Load model pre-trained parameters
input_net = "AE_net.pt"
if os.path.isfile(input_net):
    try:
        model.load_state_dict(torch.load(input_net))
        print("Training coefficients loaded successfully from file:", input_net)
    except:
        print("Error loading training coefficients from file:", input_net)
else:
    print("File", input_net, "does not exist. Training coefficients were not loaded.")

In [10]:
# Define loss and optimiziation parameters
criterion = nn.MSELoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3000, gamma=0.1)
scheduler_active_flag = True

In [11]:
print(f"Total elements in dataset: {len(dataset)}")
print(f"Shape of the first element in dataset: {dataset[0][0].shape}")
print(f"Data type of the first element in dataset: {dataset[0][0].dtype}")

Total elements in dataset: 1000
Shape of the first element in dataset: torch.Size([27104])
Data type of the first element in dataset: torch.float64


In [18]:
# Investigate data structure
for epoch in range(1):
    batch_iter = 0
    loss_tot = 0.0
    for x in data_loader:
        print(f"{len(x)} buenas {x[0].shape}")
        snapshot = x[0].type(torch.FloatTensor).to(device)
        print(snapshot.size())
        break

1 buenas torch.Size([128, 27240])
torch.Size([128, 27240])


#### Training loop

In [22]:
# Start the training loop
num_epochs = 500  # 9000
outputs = []
loss_list = []
start = time.time()
for epoch in range(num_epochs):
    batch_iter = 0
    loss_tot = 0.0

    for x in data_loader:
        # if data_loader is configured to load minibatches of data, then in each iteration of the loop, you're processing a single minibatch, rather than the entire dataset
        # x is a list originally, so we have to get the first element which is the tensor
        snapshot = x[0].type(torch.FloatTensor).to(device)
        recon = model(snapshot)
        loss = criterion(recon, snapshot)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_tot += loss.item()
        # Save results for the last epoch
        if epoch == num_epochs - 1:
            outputs.append((epoch + 1, batch_iter, snapshot, recon.detach()))
        batch_iter += 1

    # Calculate and print total average loss over all batches
    loss_tot = loss_tot / batch_iter
    loss_list.append((epoch + 1, loss_tot))
    # print(f'Epoch: {epoch+1}, Total avg loss: {loss_tot:.10f}', flush = True)
    if scheduler_active_flag:
        scheduler.step()

end = time.time()
print("Time elapsed for AE:", end - start)

Epoch: 1, Total avg loss: 0.0107995179
Time elapsed for AE: 43.00983142852783


In [26]:
# Investigate ouputs content
print("Number of elements in outputs:", len(outputs))
item = outputs[0]
epoch_num, batch_num, snapshot, recon = item
print(f"Element 0:")
print(f"Epoch: {epoch_num}, Batch: {batch_num}")
print(f"Snapshot shape: {snapshot.shape}")
print(f"Recon shape: {recon.shape}")

Number of elements in outputs: 8
Element 0:
Epoch: 1, Batch: 0
Snapshot shape: torch.Size([128, 27240])
Recon shape: torch.Size([128, 27240])


### Post-process

In [27]:
# Organize results for saving and visualization
# Unshuffle results and reconstructions
outx_shuffled = []
outxrec_shuffled = []
for i in range(int(np.ceil(m / batchsize))):
    # output[i][0,1,2,3] -> epoch_num, batch_num, snapshot, recon
    outx_shuffled.append(outputs[i][2])
    outxrec_shuffled.append(outputs[i][3])
# Concatenates  tensors from list into a single tensor
x_out_shuffled = torch.cat(outx_shuffled).detach().cpu().numpy()
xrec_out_shuffled = torch.cat(outxrec_shuffled).detach().cpu().numpy()

x_out = np.zeros(x_out_shuffled.shape)
xrec_out = np.zeros(xrec_out_shuffled.shape)

j = 0
for i in index_list:
    x_out[i, :] = x_out_shuffled[j, :]
    xrec_out[i, :] = xrec_out_shuffled[j, :]
    j += 1

In [28]:
# Reconstruction relative error
error_rec = np.linalg.norm(x_out - xrec_out) / np.linalg.norm(x_out)
# bottleneck layer (low dimensional space)
bottleneck = 8
np.savetxt(f"error_rec_{bottleneck}.txt", [error_rec], fmt="%.5e")
print("Relative reconstruction error: %.5e" % (error_rec))

# Save AE parameters
torch.save(model.state_dict(), "./AE_net" + ".pt")

Relative reconstruction error: 1.42596e-01


In [ ]:
# Save loss
df = pd.DataFrame(loss_list, columns=["Epoch", "Loss"])
df.to_csv("loss.csv", index=False)

# Plot loss as a function of the number of epochs
loss_mat = np.asarray(loss_list)
np.savetxt("loss_mat.csv", loss_mat, delimiter=",")
plt.figure(1)
plt.plot(loss_mat[:, 0], loss_mat[:, 1], linestyle="-")
plt.xlabel("Epochs")
plt.ylabel("MSE Loss")
plt.title("AE Loss")
plt.semilogy()
plt.tight_layout()
plt.savefig("AE_loss.png", dpi=200)